In [ ]:
# df1 = pd.read_csv('resume_data_improved.csv')
# df2 = pd.read_csv('resume_data_developer.csv')
# pattern = "resume_data"
# files = [f for f in os.listdir(directory) if os.path.isfile(os.path.join(directory, f)) and f.startswith(pattern)]
# df_full = [pd.read_csv(csv) for csv in files]
# df = pd.concat(df_full, axis=0, ignore_index=True).drop_duplicates()
# df.shape

In [ ]:
df.head()

In [ ]:
# df = pd.concat([df1, df2], axis=0).drop_duplicates()

In [ ]:
df.isnull().sum() / df.shape[0] * 100

In [ ]:
df.info()

In [ ]:
df = df.drop(['Availability Date', 'Date Posted', 'Unnamed: 0', 'Reference'], axis=1)

In [ ]:
df.shape

In [ ]:
df.nunique()

In [ ]:
df[['Willing to Relocate', 'U.S. Work Authorization', 'Willing to Travel']].value_counts()

In [ ]:
# FEATURE PREPROCESSING
import numpy as np, re
df['Willing to Relocate'] = df['Willing to Relocate'].map({'No': 'no', 'Yes': 'yes'})
df.loc[df['U.S. Work Authorization'] == 'Other', 'U.S. Work Authorization'] = np.nan

In [ ]:
def myfunc(a):

    if a is np.nan:
        return np.nan
    res = re.search(r'([0-9]+-)*[0-9]+%$', a)
    if res and res.group() in ['50-75%', '25%', '25-50%', '75%']:
        return res.group()[:-1]
    else:
        return a

df['Willing to Travel'] = df['Willing to Travel'].apply(myfunc)
df['Willing to Travel'] = df['Willing to Travel'].map({'50-75': 0.625, 'No': 0, '25': 0.25, '25-50': 0.375})

In [ ]:
df.info()

In [ ]:
df['Desired Industry'].value_counts().sort_values(ascending=False)

In [ ]:
df['Role'].sort_values().unique()

# START FROM HERE

In [18]:
import re, numpy as np
import os, pandas as pd
directory = os.path.join(os.getcwd())
files = [f for f in os.listdir(directory) if os.path.isfile(os.path.join(directory, f)) and re.search(r'category_[0-9]+', f)]

In [19]:
df2 = pd.concat([pd.read_csv(csv) for csv in files], axis=0, ignore_index=True).drop_duplicates()

In [20]:
df2['Desired Industry'].value_counts()

Engineering                       1500
Information Technology            1500
Accounting/Bookkeeping            1500
Business/Management               1500
Finance/Investment                1276
Computer Software/Programming     1057
Banking/Mortgage                  1007
Graphics Design/Commercial Art     604
CAD/Drafting                       556
Automotive/Mechanic                483
Computer Hardware                  330
Quality Control                    289
Public Relations                   227
Name: Desired Industry, dtype: int64

In [21]:
df2 = df2.drop(['Availability Date', 'Date Posted', 'Unnamed: 0', 'Reference', 'Affiliations', 'Additional Information'], axis=1)

In [22]:
df2.shape

(11829, 15)

In [37]:

attr = ['Skills', 'Objective', 'Education', 'Experience']
for col in attr:
    df2[col] = df2[col].apply(lambda x: str(x).encode('ascii', errors='ignore').decode().strip())
    df2[col] = df2[col].replace(r'^[-=]+[-=]$', np.nan ,  regex=True)
    df2[col] = df2[col].replace(['nan', 'non'], np.nan)
    df2[col] = df2[col].str.replace('\t', '')
    df2[col] = df2[col].str.replace('\n', '.')

0        Bachelor of Arts in Geosciences  University of...
1        Philippines Licensed Mechanical Engineer with ...
2        Bachelor of Technology in Mechanichal Engineer...
3        Bachelor of Engineering Mechanical Engineering...
4        Hands-on, analytical, and solutions-focused pr...
                               ...                        
11824                                                  NaN
11825                                                  NaN
11826                                                  NaN
11827                                                  NaN
11828                                                  NaN
Name: Education, Length: 11829, dtype: object

In [ ]:
df2.columns

In [ ]:
df2['Desired Industry'].value_counts()

In [ ]:
df2.Skills

In [ ]:
df2.shape

In [46]:
df2 = df2.fillna('')

In [ ]:
# df2.to_csv('data.csv')

In [ ]:
df['Skills'] + ' '+ df['Objective']

In [47]:
df2.isnull().sum()

Role                       0
Desired Industry           0
SpiderID                   0
Desired Job Location       0
Type of Position           0
Desired Wage               0
U.S. Work Authorization    0
Job Level                  0
Willing to Travel          0
Highest Degree Attained    0
Willing to Relocate        0
Objective                  0
Experience                 0
Education                  0
Skills                     0
dtype: int64

# **NLP PART**

In [ ]:
import spacy

In [49]:
nlp = spacy.load('en_core_web_sm')

In [50]:
df_copy = df2.copy()

In [42]:
def preprocess_doc(doc):
    # Tokenize the document
    # tokens = [token.text for token in doc]

    # Remove stop words
    tokens = [token for token in doc if not token.is_stop]

    # Lemmatize the tokens
    tokens = [token.lemma_ for token in doc]

    # Join the tokens back into a single string
    preprocessed_doc = ' '.join(tokens)

    return preprocessed_doc

In [ ]:
for col in ['Skills', 'Objective', 'Education', 'Experience']:
    df_copy[col] = df_copy[col].apply(lambda x: preprocess_doc(nlp(x)))

In [ ]:
df_copy['Skills2'] = df_copy['Skills'].apply(lambda x: nlp(x))

In [ ]:
df_copy['Skills2']

In [ ]:
doc = preprocess_doc(nlp(df_copy.loc[3, 'Experience']))

In [ ]:
doc